In [1]:
!pip install annoy
import os
import pandas as pd
import numpy as np

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for annoy: filename=annoy-1.17.3-cp310-cp310-linux_x86_64.whl size=552446 sha256=6f54fcb64c38a70e784e3c65c9add4c267ec8fa505f1e4a7473de9212a1411cd
  Stored in directory: /root/.cache/pip/wheels/64/8a/da/f714bcf46c5efdcfcac0559e63370c21abe961c48e3992465a
Successfully built annoy


In [2]:
!gdown https://drive.google.com/uc?id=1JWTqMEiUZ2yNUJJl_5Ctq8SuskVocn51&export=download
!tar -xf CACD2000_refined.tar

Downloading...
From: https://drive.google.com/uc?id=1JWTqMEiUZ2yNUJJl_5Ctq8SuskVocn51
To: /content/CACD2000_refined.tar
100% 1.99G/1.99G [00:30<00:00, 66.1MB/s]


In [3]:
!gdown --id 1HFxHX2RkEr7_yVHnA-qk5Lj8CxOWrUda

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1HFxHX2RkEr7_yVHnA-qk5Lj8CxOWrUda
To: /content/final_embeddings_clusters.parquet.gzip
100% 1.85G/1.85G [00:42<00:00, 43.3MB/s]


In [4]:
df = pd.read_parquet("final_embeddings_clusters.parquet.gzip")

In [5]:
from annoy import AnnoyIndex

f = len(df['embedding'][0])
t = AnnoyIndex(f, metric='euclidean')
# ntree = 50

# for i, vector in enumerate(df['embedding']):
#     t.add_item(i, vector)
# _  = t.build(ntree)

t.load('/content/CACD2000_refined_image_embeddings_clusters.ann')

In [12]:
t.save('CACD2000_refined_image_embeddings_clusters.ann')

True

In [9]:
def get_similar_images_annoy(img_index, n=10, max_dist=10):
    vid, face  = df.iloc[img_index, [0, 1]]
    # print(vid, face)
    similar_img_ids, dist = t.get_nns_by_item(img_index, n+1, include_distances=True)

    temp = similar_img_ids[::-1]
    dtemp = dist[::-1]

    t1 = []

    for s,d in zip(temp, dtemp):
      #if d >= 0.3:
      #print(d)
      t1.append(s)

    similar_img_ids = t1[1:]

    #similar_img_ids = [s for s,d in zip(similar_img_ids, dist) if d <= max_dist][1:]  # first item is always its own video
    return vid, vid, df.iloc[similar_img_ids], dist

In [10]:
  def get_sample_n_similar(sample_idx):
      output_images = []
      vid, face, similar, distances = get_similar_images_annoy(sample_idx)

      #fig = plt.figure(figsize=(15, 7))
      #gs = fig.add_gridspec(2, 6)
      #ax1 = fig.add_subplot(gs[0:2, 0:2])
      #ax2 = fig.add_subplot(gs[0, 2])
      #ax3 = fig.add_subplot(gs[0, 3])
      #ax4 = fig.add_subplot(gs[0, 4])
      #ax5 = fig.add_subplot(gs[0, 5])
      #ax6 = fig.add_subplot(gs[1, 2])
      #ax7 = fig.add_subplot(gs[1, 3])
      #ax8 = fig.add_subplot(gs[1, 4])
      #ax9 = fig.add_subplot(gs[1, 5])
      #axx = [ax1, ax2, ax3, ax4, ax5, ax6, ax7, ax8, ax9]
      #for ax in axx:
      #    ax.set_axis_off()
      list_plot = [face] + similar['face'].values.tolist()
      list_cluster = [df.iloc[sample_idx]['cluster']] + similar['cluster'].values.tolist()
      for face, cluster, dist in zip(list_plot, list_cluster, distances):
        try:
          output_images.append(f'{face.split("/")[-1][:-4]}.jpg')
        except:
          continue
          # print(face)
          # print("did not exist")
      return output_images

In [11]:
test = []
for i in np.random.choice(len(df), 3, replace=False):
    test.append(get_sample_n_similar(i))
    #get_similar_images_annoy(i)
    #print(i)

#get_sample_n_similar(i)
test

[['58_Michael_Moore_0002.jpg',
  '50_Michael_Moore_0001.jpg',
  '52_Michael_Moore_0006.jpg',
  '50_Michael_Moore_0010.jpg',
  '56_Michael_Moore_0014.jpg',
  '59_Michael_Moore_0004.jpg',
  '59_Michael_Moore_0010.jpg',
  '57_Michael_Moore_0005.jpg',
  '58_Michael_Moore_0010.jpg',
  '58_Michael_Moore_0003.jpg',
  '58_Michael_Moore_0002.jpg'],
 ['25_Lena_Dunham_0001.jpg',
  '30_Jayma_Mays_0007.jpg',
  '43_Kelly_Preston_0001.jpg',
  '30_Isla_Fisher_0004.jpg',
  '30_Maggie_Grace_0001.jpg',
  '33_Jenna_Fischer_0003.jpg',
  '24_Nina_Dobrev_0016.jpg',
  '22_Lena_Dunham_0014.jpg',
  '24_Sara_Canning_0013.jpg',
  '22_Lena_Dunham_0009.jpg',
  '25_Lena_Dunham_0001.jpg'],
 ['56_Tanya_Roberts_0015.jpg',
  '50_Beth_Broderick_0008.jpg',
  '27_Brittany_Snow_0005.jpg',
  '28_A.J._Cook_0005.jpg',
  '60_Kate_Capshaw_0009.jpg',
  '21_Stephanie_Leonidas_0004.jpg',
  '19_Caleb_Landry_Jones_0009.jpg',
  '27_Michael_Raymond-James_0010.jpg',
  '44_Patricia_Arquette_0005.jpg',
  '48_Steve_Coogan_0004.jpg',
  '56_